# 4. One-Hot Encoding

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge  # placeholder model
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Spotify_cleaned_stage3.csv')
df.head()

,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,popularity_class,release_year,release_month
0,6f807x0ima9a1j3VPbc7VN,I Dont Care with Justin Bieber Loud Luxury Remix,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Dont Care with Justin Bieber Loud Luxury Remix,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754,60–80,2019,6
1,0r7CVbZTWZgbTCYdfa2P31,Memories Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories Dillon Francis Remix,2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600,60–80,2019,12
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time Don Diablo Remix,2019-05-07,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616,60–80,2019,5
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093,40–60,2019,7
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved Future Humans Remix,2019-05-03,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052,60–80,2019,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29729 entries, 0 to 29728
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  29729 non-null  object 
 1   track_name                29719 non-null  object 
 2   track_artist              29721 non-null  object 
 3   track_popularity          29729 non-null  int64  
 4   track_album_id            29729 non-null  object 
 5   track_album_name          29715 non-null  object 
 6   track_album_release_date  29729 non-null  object 
 7   playlist_name             29729 non-null  object 
 8   playlist_id               29729 non-null  object 
 9   playlist_genre            29729 non-null  object 
 10  playlist_subgenre         29729 non-null  object 
 11  danceability              29729 non-null  float64
 12  energy                    29729 non-null  float64
 13  key                       29729 non-null  int64  
 14  loudne

In [5]:
# ==============================================================
# Label Encoding + Naive Bayes
# ==============================================================

# --------------------------------------------------------------
# Select categorical features to encode (moderate cardinality only)
# --------------------------------------------------------------
cat_cols = []
for c in ['playlist_genre', 'playlist_subgenre', 'release_year', 'release_month']:
    if c in df.columns:
        df[c] = df[c].astype(str)
        cat_cols.append(c)

# Drop any rows missing the target class
df = df.dropna(subset=['popularity_class'])

# Apply LabelEncoder to selected categorical columns
encoder = LabelEncoder()
for c in cat_cols:
    df[c] = encoder.fit_transform(df[c])

# Safety check: which columns are still non-numeric?
non_numeric = [c for c in df.columns if df[c].dtype == 'object']
print("Non-numeric columns remaining:", non_numeric)

# Define features (X) as NUMERIC ONLY and drop IDs/target
id_like = ['track_id','track_album_id','playlist_id']
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

X = df[num_cols].drop(columns=['track_popularity'], errors='ignore')  # keep encoded cats (now numeric)
# y stays categorical labels
y = df['popularity_class']


# Train/Test Split (stratified)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X.drop(columns=id_like, errors='ignore'),
    y,
    test_size=0.2, random_state=42, stratify=y
)

# Train Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Evaluate
from sklearn.metrics import accuracy_score, classification_report
y_pred = gnb.predict(X_test)
print(f"\n🎯 Gaussian NB Accuracy: {accuracy_score(y_test, y_pred):.3f}\n")
print(classification_report(y_test, y_pred))



Non-numeric columns remaining: ['track_id', 'track_name', 'track_artist', 'track_album_id', 'track_album_name', 'track_album_release_date', 'playlist_name', 'playlist_id']

🎯 Gaussian NB Accuracy: 0.317

              precision    recall  f1-score   support

       20-40       0.33      0.07      0.12      1214
       40-60       0.32      0.80      0.46      1758
       60-80       0.29      0.15      0.20      1423
         <20       0.23      0.02      0.04       825
         >80       0.00      0.00      0.00       255

    accuracy                           0.32      5475
   macro avg       0.23      0.21      0.16      5475
weighted avg       0.29      0.32      0.23      5475



# ✅Finished One-Hot Encoding, on to Feature Engineering & Feature Selection